# Using predictors trained on DMSexp and EVmutation datasets

This vignette shows how to prepare variants for applying our predictors trained on DMSexp and EVmutation datasets. 

For illustration we will use the BRCA1 DMS experimental dataset by [Findlay et al Nature 2018](https://www.nature.com/articles/s41586-018-0461-z). The code (python and R) below shows how to annotate the features required for the predictors, and applying the classifiers. We will have the actual experimental data, as well as predictions offered by [EVmutation](https://marks.hms.harvard.edu/evmutation/), to compare against.


In [1]:
%load_ext rpy2.ipython

In [2]:
import pickle
import numpy as np
import pandas as pd
import sklearn

In [3]:
BRCA_DMS = pd.read_csv('Findlay_et_al_BRCA_DMS.txt', sep = '\t', skiprows = 2)
BRCA_DMS = BRCA_DMS[BRCA_DMS['func.class'] != 'None']
BRCA_EVmutation = pd.read_csv("BRCA1_EVmutation.tsv", sep = "\t")
BRCA_DMS['aa_pos'] = BRCA_DMS['aa_pos'].astype(str)
BRCA_EVmutation['pos'] = BRCA_EVmutation['pos'].astype(str)
BRCA_DMS['function.score.mean'] = BRCA_DMS['function.score.mean'].astype(float)

In [4]:
BRCA_DMS = pd.merge( BRCA_DMS, BRCA_EVmutation.drop_duplicates(), \
                     left_on = ["aa_pos", "aa_ref", "aa_alt"], \
                     right_on = ["pos", "wt", "subs"])
BRCA_DMS

,aa_pos,aa_ref,aa_alt,function.score.mean,func.class,uniprot,mutant,pos,wt,subs,prediction_epistatic
0,21,I,L,0.244698,0,P38398,I21L,21,I,L,-0.9572
1,21,I,V,-0.147346,0,P38398,I21V,21,I,V,-0.9709
2,21,I,F,-0.432699,0,P38398,I21F,21,I,F,-2.5595
3,21,I,N,-0.215463,0,P38398,I21N,21,I,N,1.5518
4,21,I,T,0.072621,0,P38398,I21T,21,I,T,-0.0365
...,...,...,...,...,...,...,...,...,...,...,...
688,1731,E,A,0.275906,0,P38398,E1731A,1731,E,A,-5.8658
689,1731,E,G,0.065227,0,P38398,E1731G,1731,E,G,-8.0551
690,1731,E,V,-0.203848,0,P38398,E1731V,1731,E,V,-8.0551
691,1731,E,D,0.125433,0,P38398,E1731D,1731,E,D,-6.8724


## Obtaining variant annotation

The first step is to annotate the variants such that we have the features that are needed for the predictors to work. Specifically, we need the following features:

1. **DNA trinucleotide mutational signature**: The [CDSMutSig](https://github.com/josef0731/CDSMutSig) R package provides functionalities to obtain this.
2. **DNA Conservation scores** of the trinucleotide signature: From (1), we can simply query these scores (using packages such as `genomicScores` in R/Bioconductor) with the genomic coordinates mapped.
3. **Solvent accessibility** information from mapping to structures: We use mappings provided in the [ZoomVar](https://fraternalilab.kcl.ac.uk/ZoomVar/) database. Mapping per amino acid position to structures from ZoomVar is included as a flat-file data table in the [Zenodo repository of this manuscript](https://doi.org/10.5281/zenodo.4726168).


### 1. DNA Mutational signature

the following code snippet in R shows how to use CDSMutSig to obtain mappings for given protein(s) - it gives all missense variants which can be obtained by substituting one nucleotide, at any positions along the protein sequence:

In [5]:
%%R -o mutsigs
# Map genomic position, MutSig and phyloP scores

ensp <- data.frame(
  uniprot_id = c("P38398"),
  protein_id = c("ENSP00000350283"), # manually checked by searching UniProt & Ensembl 
  stringsAsFactors = FALSE
)

# Use CDSMutSig
library(CDSMutSig)
mutsigs <- lapply(ensp$protein_id, function(id){
  cat(paste0(id, " ...\n"))
  codons <- mapCodon(id, EnsDb.Hsapiens.v86::EnsDb.Hsapiens.v86,
                     BSgenome.Hsapiens.NCBI.GRCh38::Hsapiens, 
                     seqLevelStyle = "NCBI")
  if( is.null( codons ) ) return(NULL)
  codons_muts <- mapMut(codons, getPossibleMissenseSNVs())
  codons_mutsigs <- mapMutSig(id, nrow(codons), codons_muts, 
                              EnsDb.Hsapiens.v86::EnsDb.Hsapiens.v86,
                              BSgenome.Hsapiens.NCBI.GRCh38::Hsapiens, 
                              seqLevelStyle = "NCBI")
  codons_mutsigs$ensp <- id  
  codons_mutsigs
})
       
mutsigs <- do.call("rbind", mutsigs)
mutsigs <- merge( ensp, mutsigs, by.x = "protein_id", by.y = "ensp" )

ENSP00000350283 ...


R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK

R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK



In [6]:
mutsigs

,protein_id,uniprot_id,chr,g_pos,AA_pos,WT_AA,MUT_AA,WT_codon,MUT_codon,MutSig
1,ENSP00000350283,P38398,17,43124018.0,1,M,T,ATG,ACG,A[T>C]G
2,ENSP00000350283,P38398,17,43124017.0,1,M,L,ATG,TTG,A[T>A]T
3,ENSP00000350283,P38398,17,43124018.0,1,M,K,ATG,AAG,A[T>A]G
4,ENSP00000350283,P38398,17,43124018.0,1,M,R,ATG,AGG,A[T>G]G
5,ENSP00000350283,P38398,17,43124017.0,1,M,L,ATG,CTG,A[T>G]T
...,...,...,...,...,...,...,...,...,...,...
13271,ENSP00000350283,P38398,17,43045802.0,1863,Y,*,TAC,TAG,A[C>G]T
13272,ENSP00000350283,P38398,17,43045802.0,1863,Y,*,TAC,TAA,A[C>A]T
13273,ENSP00000350283,P38398,17,43045800.0,1863,Y,D,TAC,GAC,C[T>G]A
13274,ENSP00000350283,P38398,17,43045801.0,1863,Y,F,TAC,TTC,G[T>A]A


As shown above, this gives a full table of all possible missense variants, their genomic positions, and the corresponding trinucleotide signature (column `MutSig`).

### 2. Conservation

With the `chr` and `g_pos` columns above we can fetch the conservation scores of the trinucleotide motif surrounding the substitution. Here we use the genomicScores R package to fetch the phyloP score of the trinucleotide positions:

In [7]:
%%R -i mutsigs -o PhyloP

# Use PhyloP                                  
library(GenomicRanges)
library(GenomicScores)
library(IRanges)
library(stringr)

phast <- getGScores("phyloP100way.UCSC.hg38")

PhyloP <- sapply(-1:1, function(x){
  gscores_phylop <- gscores(phast,
        GenomicRanges::GRanges(seqnames = paste0("chr", mutsigs[, "chr"]),
                               ranges = IRanges::IRanges(
                               start = mutsigs[, "g_pos"] + x,
                               end = mutsigs[, "g_pos"] + x)))
  gscores_phylop <- as.data.frame(gscores_phylop)
  return( gscores_phylop[, 6] )
})

colnames( PhyloP ) <- paste("PhyloP", -1:1, sep = "_")
PhyloP <- as.data.frame(PhyloP)


/home/josefng/.local/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:59: UserWarning: Error while trying to convert the column "protein_id". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  warnings.warn('Error while trying to convert '
/home/josefng/.local/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:59: UserWarning: Error while trying to convert the column "uniprot_id". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  warnings.warn('Error while trying to convert '
/home/josefng/.local/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:59: UserWarning: Error while trying to convert the column "chr". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  warnings.warn('Error while trying to convert '
/home/josefng/.local/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:59: UserWarning: Error while trying to convert the column "WT_AA". Fall back to string c

In [8]:
mutsigs = pd.concat([mutsigs, PhyloP], axis = 1)
mutsigs['AA_pos'] = mutsigs['AA_pos'].astype(str)
mutsigs

,protein_id,uniprot_id,chr,g_pos,AA_pos,WT_AA,MUT_AA,WT_codon,MUT_codon,MutSig,PhyloP_-1,PhyloP_0,PhyloP_1
1,ENSP00000350283,P38398,17,43124018.0,1,M,T,ATG,ACG,A[T>C]G,4.0,4.0,0.5
2,ENSP00000350283,P38398,17,43124017.0,1,M,L,ATG,TTG,A[T>A]T,4.0,4.0,4.0
3,ENSP00000350283,P38398,17,43124018.0,1,M,K,ATG,AAG,A[T>A]G,4.0,4.0,0.5
4,ENSP00000350283,P38398,17,43124018.0,1,M,R,ATG,AGG,A[T>G]G,4.0,4.0,0.5
5,ENSP00000350283,P38398,17,43124017.0,1,M,L,ATG,CTG,A[T>G]T,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13271,ENSP00000350283,P38398,17,43045802.0,1863,Y,*,TAC,TAG,A[C>G]T,-0.5,0.0,4.0
13272,ENSP00000350283,P38398,17,43045802.0,1863,Y,*,TAC,TAA,A[C>A]T,-0.5,0.0,4.0
13273,ENSP00000350283,P38398,17,43045800.0,1863,Y,D,TAC,GAC,C[T>G]A,4.0,3.0,-0.5
13274,ENSP00000350283,P38398,17,43045801.0,1863,Y,F,TAC,TTC,G[T>A]A,3.0,-0.5,0.0


The columns `PhyloP_-1`, `PhyloP_0`, `PhyloP_1` refer to the phyloP conservation scores at position -1 (i.e. 1 base 5' to the DNA substitution), 0 (i.e. the substituted DNA position) and +1 (i.e. 1 base 3' to the DNA substitution), for each missense variant annotated using CDSMutSig.

### 3. Solvent accessibility

We use mappings provided in the ZoomVar database, which in turns use the [POPS and POPSComp](https://github.com/Fraternalilab/POPScomp) programs to annotate the solvent accessibility of each residue in the protein structure. The full table containing all analysed human Uniprot entries can be found in the [Zenodo repository of this manuscript](https://doi.org/10.5281/zenodo.4726168).

In [9]:
# the zoomvar Q(SASA) information
zoomvar = pd.read_csv('BRCA1_ZoomVar.tsv', sep = "\t", index_col=False)
zoomvar = zoomvar[['protein', 'position', 'Q(SASA)']]
zoomvar['position'] = zoomvar['position'].astype(str)
zoomvar

,protein,position,Q(SASA)
0,P38398,1,None
1,P38398,2,None
2,P38398,3,None
3,P38398,4,None
4,P38398,5,None
...,...,...,...
1858,P38398,1859,0.9639
1859,P38398,1860,None
1860,P38398,1861,None
1861,P38398,1862,None


Here we merge the tables together. We will only retain those variants with all the required mappings, i.e. only variants mappable to structures & manifested as single nucleotide substitution.

In [10]:
# merge tables
BRCA_DMS = pd.merge( BRCA_DMS, mutsigs, \
                     left_on = ["aa_pos", "aa_ref", "aa_alt"], \
                     right_on = ["AA_pos", "WT_AA", "MUT_AA"])

BRCA_DMS = pd.merge( BRCA_DMS, zoomvar, left_on = ['uniprot_id', 'AA_pos'], right_on = ['protein', 'position'])
BRCA_DMS = BRCA_DMS[ BRCA_DMS['Q(SASA)'].str.find('None') == -1 ]

BRCA_DMS

,aa_pos,aa_ref,aa_alt,function.score.mean,func.class,uniprot,mutant,pos,wt,subs,...,MUT_AA,WT_codon,MUT_codon,MutSig,PhyloP_-1,PhyloP_0,PhyloP_1,protein,position,Q(SASA)
0,21,I,L,0.244698,0,P38398,I21L,21,I,L,...,L,ATC,CTC,A[T>G]T,-3.0,-3.0,1.5,P38398,21,0.3845
1,21,I,V,-0.147346,0,P38398,I21V,21,I,V,...,V,ATC,GTC,A[T>C]T,-3.0,-3.0,1.5,P38398,21,0.3845
2,21,I,F,-0.432699,0,P38398,I21F,21,I,F,...,F,ATC,TTC,A[T>A]T,-3.0,-3.0,1.5,P38398,21,0.3845
3,21,I,N,-0.215463,0,P38398,I21N,21,I,N,...,N,ATC,AAC,A[T>A]C,-3.0,1.5,2.0,P38398,21,0.3845
4,21,I,T,0.072621,0,P38398,I21T,21,I,T,...,T,ATC,ACC,A[T>C]C,-3.0,1.5,2.0,P38398,21,0.3845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,1731,E,V,-0.203848,0,P38398,E1731V,1731,E,V,...,V,GAG,GTG,C[T>A]C,1.0,5.0,5.0,P38398,1731,0.1122
881,1731,E,D,0.125433,0,P38398,E1731D,1731,E,D,...,D,GAG,GAT,C[C>A]T,5.0,5.0,5.0,P38398,1731,0.1122
882,1731,E,D,0.125433,0,P38398,E1731D,1731,E,D,...,D,GAG,GAC,C[C>G]T,5.0,5.0,5.0,P38398,1731,0.1122
883,1731,E,D,-0.031138,0,P38398,E1731D,1731,E,D,...,D,GAG,GAT,C[C>A]T,5.0,5.0,5.0,P38398,1731,0.1122


In [11]:
BRCA_DMS.drop_duplicates().to_csv("BRCA1_DMS_annotated.csv")

We now have all the features required for the predictors. Functions in  below process the features:

1. parse DNA substitution type & base at position -1 and +1 separately into three features
2. Normalise PhyloP score so they are in range [-1, 1]
3. Scale Q(SASA) by subtracting 0.15. So that +ve value means surface (exposed) and -ve value means core (buried)
4. One-hot encoding of the non-numeric features.

In [12]:
from helper import * # functions in helper.R in this repository

X, Y = prepare_data('BRCA1_DMS_annotated.csv', delimiter=",", output_col = "func.class")
# save the objects
pickle.dump( (X, Y), open('BRCA1_DMS_annotated.pickle', 'wb') )

Data read in. Now process them
PhyloP score normalised to [-1, 1].
MutSig parsed.
WT_AA one-hot encoding ...
MUT_AA one-hot encoding ...
-1 and +1 DNA pos one-hot encoding ...
DNA substitution one-hot encoding ...


## Apply the predictors

Now we can apply the predictors. We have predictors trained on two datasets:

1. `DMSexp`: trained on experimental DMS data covering 10 proteins.
2. `EVmutation`: trained on computational DMS data given by a scoring system implemented in [EVmutation](https://marks.hms.harvard.edu/evmutation/).

For each, we have trained predictors using these different combinations of features:

* AA + Cons (i.e. Conservation) + MutSig + QSASA (i.e. Solvent accessibility)
* AA + Cons + MutSig
* AA + MutSig + QSASA
* AA + MutSig
* MutSig only
* AA only

Additionally, for `DMSexp` and `EVmutation`, we also trained a version of predictors where variants residing in the protein core were removed from training, to evaluate the importance of protein core positions to learning variant effects. We call these the *no-core* predictors.

All predictors were trained using the Gradient Boosting Classifier method in scikit-learn, and can be loaded from the pickle objects included in the repository - see below for example.

Here we apply the predictors on the BRCA1 variants and show the ROC-Area-under-curve (AUC) - a higher ROC-AUC signifies better prediction performance.

### DMSexp

In [13]:
setups = {"AA_Cons_MutSig_QSASA": range(58), "AA_Cons_MutSig": range(57), \
          "AA_MutSig_QSASA": [i for i in range(54)] + [57], "AA_MutSig": range(54), \
          "AA": range(40), "MutSig": range(41, 54)}

DMSexp = dict()

for setup, col_id in setups.items():
    with open('DMSexp/setup/' + setup + '/GBM.pickle', 'rb' ) as r:
        model = pickle.load( r )
        DMSexp[setup] = sklearn.metrics.roc_auc_score(Y, model.predict_proba( X[:, col_id] )[:, 1])

pd.DataFrame.from_dict(DMSexp, orient='index', columns=['ROC-AUC'])

,ROC-AUC
AA_Cons_MutSig_QSASA,0.668973
AA_Cons_MutSig,0.606973
AA_MutSig_QSASA,0.651754
AA_MutSig,0.727465
AA,0.767858
MutSig,0.623852


### EVmutation

The original EVmutation training set contains BRCA1 variants, therefore uses the version trained on variants excluding BRCA1.

In [14]:
EVmutation = dict()

for setup, col_id in setups.items():
    with open('EVmutation/setup/' + setup + '/GBM-excludeBRCA1-PTEN.pickle', 'rb' ) as r:
        model = pickle.load( r )
        EVmutation[setup] = sklearn.metrics.roc_auc_score(Y, model.predict_proba( X[:, col_id] )[:, 1])

pd.DataFrame.from_dict(EVmutation, orient='index', columns=['ROC-AUC'])

,ROC-AUC
AA_Cons_MutSig_QSASA,0.849537
AA_Cons_MutSig,0.775805
AA_MutSig_QSASA,0.847734
AA_MutSig,0.760578
AA,0.768005
MutSig,0.555248
